# Integración numérica de la ecuación de onda
Utilizaré distintos métodos (pues el profe hinteó que los algorimos usuales no van a funcionar, pero quisiera ver qué es lo que le pasa) para integrar numéricamente la ecuación de onda adimensionalizada:
$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x)
$$


## Algoritmo naive:

La idea es usar la clásica discretización de la segunda derivada a ambos lados:

$$
\partial_{zz} f(z) \approx \frac{f(z + h) - 2 f(z) + f(z - h)}{h^2} 
$$

Y con esto, despejar el valor de $u(t + \Delta t, x)$ en función de valores de tiempos anteriores y otras posiciones. Para el primer paso de tiempo simplemente usaré la condición inicial, es decir, supongo velocidad inicial 0 para cada $u(\cdot, x)$. También usaré condiciones de borde fijas.


In [33]:
using LinearAlgebra
using GLMakie

const x_largo = 1.0
const N_x = 201
const Δx = x_largo/(N_x - 1)
const Δt = Δx
const s = (Δt/Δx)^2
println("s = ", s)
const sMatrix = Tridiagonal(ones(N_x-3) .* s, ones(N_x-2) .* 2 .* (1 - s), ones(N_x-3) .* s)
const bordes = 0.0
const x = range(0, x_largo, length=N_x)

s = 1.0


0.0:0.005:1.0

In [34]:
function paso_temporal!(u, u_anterior)
    buffer = copy(u)
    u[2:end-1] .= sMatrix * u[2:end-1] .- u_anterior[2:end-1]
    u[1] = bordes
    u[end] = bordes # En verdad esto no es necesario.
    u[2] += s * bordes 
    u[end-1] += s * bordes
    u_anterior .= buffer
end

function actualizar_animacion!(i)
    lines[1][] = Point2f[[x[i], u[i]] for i in 1:N_x] # Horrible sintaxis.
    println("Energía = ", energía(u, u_anterior))
    paso_temporal!(u, u_anterior)
    return
end

function energía(u, u_anterior)
    dudt = (u - u_anterior)/Δt 
    dudx = (u[2:end] - u[1:end-1])/Δx # Aproximaciones charchas goo 
    return (sum(dudt.^2).*Δx .+ sum(dudx.^2).*Δx)*0.5
end


energía (generic function with 1 method)

In [35]:
# Animación:

u = sin.(2π .* x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1])
lines = lines!(ax, x, u)
nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "modo_normal.mp4", 1:nframes; framerate=framerate)


Energía = 9.86879268536886
Energía = 9.859062989982258
Energía = 9.84938130477835
Energía = 9.839785838945485
Energía = 9.830314461403571
Energía = 9.82100455135269
Energía = 9.811892850754525
Energía = 9.8030153193286
Energía = 9.794406992635688
Energía = 9.786101843808485
Energía = 9.778132649475136
Energía = 9.77053086040491
Energía = 9.763326477386338
Energía = 9.756547932827822
Energía = 9.750221978547911
Energía = 9.74437358019809
Energía = 9.739025818734778
Energía = 9.734199799329305
Energía = 9.72991456807546
Energía = 9.726187036823273
Energía = 9.723031916435652
Energía = 9.72046165873132
Energía = 9.718486407343203
Energía = 9.717113957686125
Energía = 9.716349726191869
Energía = 9.716196728933
Energía = 9.71665556971979
Energía = 9.717724437717287
Energía = 9.719399114591832
Energía = 9.721672991158874
Energía = 9.724537093466441
Energía = 9.727980118211208
Energía = 9.731988477347489
Energía = 9.73654635171301
Energía = 9.74163575345994
Energía = 9.747236597044717
Energía

"modo_normal.mp4"

Ahora otra condición inicial

In [36]:
function u_func(x)
    if x < 1/3 || x > 2/3
        return 0.0
    elseif x < 1/2
        return x - 1/3
    else
        return 2/3 - x
    end
end


u = u_func.(x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1])
lines = lines!(ax, x, u)

nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "triángulo.mp4", 1:nframes; framerate=framerate)


Energía = 0.1655555555555555
Energía = 0.1761111111111109
Energía = 0.18666666666666634
Energía = 0.19722222222222174
Energía = 0.20777777777777695
Energía = 0.21833333333333213
Energía = 0.2288888888888873
Energía = 0.23944444444444207
Energía = 0.24999999999999672
Energía = 0.26055555555555143
Energía = 0.2711111111111059
Energía = 0.2816666666666603
Energía = 0.2922222222222147
Energía = 0.30277777777776915
Energía = 0.31333333333332347
Energía = 0.3238888888888779
Energía = 0.33444444444443233
Energía = 0.3583333333333203
Energía = 0.37555555555554165
Energía = 0.392777777777763
Energía = 0.4099999999999842
Energía = 0.4272222222222055
Energía = 0.4444444444444269
Energía = 0.46166666666664813
Energía = 0.4788888888888694
Energía = 0.49611111111109063
Energía = 0.5133333333333119
Energía = 0.530555555555533
Energía = 0.5477777777777543
Energía = 0.5649999999999755
Energía = 0.5822222222221967
Energía = 0.5994444444444178
Energía = 0.616666666666639
Energía = 0.6338888888888603
Ener

"triángulo.mp4"

Adjuntaré un análisis de estabilidad que hice, que me asegura que cuando $ s\leq 1$ el método es estable. Sin embargo, también vemos que para la condición inicial triangular no sucede que se vaya rearmando cada cierto número de oscilaciones. Esto sólo puede ocurrir si la relación de dispersión numérica es distinta a la exacta (que es $\omega = k$). Vemos que hay unas oscilaciones extrañas ahí metidas cuando $s = 1$.

## Ahora un método más avanzado

Usaré la librería FEniCS para resolver la ecuación de onda. Para esto crearé otro .ipynb, pues utiliza python (no Julia).